In [154]:
import numpy as np
import pandas as pd
import pandas_profiling
import math
import re

In [161]:
def is_number(s):
    """ Returns True is string is a number. """
    try:
        float(s)
        return True
    except ValueError:
        return False

In [162]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

### Data Cleaning

Convert BMI to numeric

In [168]:
train["BMI"] = pd.to_numeric(train["BMI"],errors='coerce')

Treat '.,' as Missing Values

In [172]:
train[train["high_BP"] == '.,'] = None

In [164]:
def clean_sex_age(sex_age_list):
    if type(sex_age_list) is not list:
        return ["",""]
    # Strip and Upper case both sex and age
    sex_age_list[0],sex_age_list[1] = sex_age_list[0].strip().upper(), sex_age_list[1].strip().upper()
    
    # 2nd : first one is empty and second one is not numeric
    if (is_number(sex_age_list[0]) or (not sex_age_list[0] and not is_number(sex_age_list[1]))): 
        sex_age_list = sex_age_list[::-1]
    sex = sex_age_list[0].strip().upper()
    
    # Take
    if sex in ('FEMALE','FEMALLE'):
        sex = 'F'
    if sex in ('MALE','MMALE','MM'):
        sex = 'M'

    sex_age_list[0] = sex
    sex_age_list[1] = sex_age_list[1].strip()
    return sex_age_list
train["sex_age_list"] = train["sex and age"].str.split(",").apply(clean_sex_age)

In [165]:
train[['sex','age']] = pd.DataFrame(train["sex_age_list"].values.tolist(), index= train.index)
train["age"] = pd.to_numeric(train["age"],errors='coerce')
train.drop(columns=["sex_age_list","sex and age"],inplace=True)
train["sex"].value_counts()

F        20557
M        14271
            37
OTHER        7
Name: sex, dtype: int64

In [3]:
def is_job_status_and_living_area_reversed(x):
    return ((x["job_status"] != None and x["job_status"] in ("r", "c", "city", "remote", "remotee"))
             or (x["living_area"] != None and x["living_area"] in ("private_sector", "business_owner")))
def process_job_status(x):
    if x == None or x in ("nan", 'null', "", 'n.a'):
        return None
    elif x in ("private sector", "privattte", "private", "private_sector"):
        return "private_sector"
    elif x in ("government", "govt."):
        return "government"
    elif x in ("business_owner", "business owner", "biz"):
        return "business_owner"
    elif x in ("parental_leave", "parental leave"):
        return "parental_leave"
    else:
        return x
    
def process_living_area(x):
    if x == None or x in ("nan", 'null', "", 'n.a'):
        return None
    elif x == 'c':
        return 'city'
    elif x in ('r', 'remotee'):
        return 'remote'
    else:
        return x

def split_job_status_and_living_area(x):
    pair = x.lower().split("?") if x != None else [x, x]
    if len(pair) < 2:
        pair = [pair[0], None]
    return pair

def process_job_status_and_living_area(df):
    df["job_status"] = df["job_status and living_area"].astype(str).apply(split_job_status_and_living_area).apply(lambda x: x[0])
    df["living_area"] = df["job_status and living_area"].astype(str).apply(split_job_status_and_living_area).apply(lambda x: x[1])
    job_status = df.apply(lambda x: x["living_area"] if is_job_status_and_living_area_reversed(x) else x["job_status"], 1)
    living_area = df.apply(lambda x: x["job_status"] if is_job_status_and_living_area_reversed(x) else x["living_area"], 1)
    df["job_status"] = job_status.apply(lambda x: process_job_status(x))
    df["living_area"] = living_area.apply(lambda x: process_living_area(x))

In [4]:
process_job_status_and_living_area(train)

In [5]:
process_job_status_and_living_area(test)

In [6]:
def process_smoker_status(x):
    if x == None:
        return None
    elif x.startswith("non"):
        return "non-smoker"
    elif x.startswith("quit"):
        return "quit"
    elif x.startswith("active"):
        return "active_smoker"
    else:
        return None
train["smoker_status"] = train["smoker_status"].astype(str).apply(process_smoker_status)
test["smoker_status"] = test["smoker_status"].astype(str).apply(process_smoker_status)

In [7]:
train.head()

,id,sex and age,high_BP,heart_condition_detected_2017,married,job_status and living_area,average_blood_sugar,BMI,smoker_status,TreatmentA,TreatmentB,TreatmentC,TreatmentD,stroke_in_2018,job_status,living_area
0,16053,"F, 61",1,0,1,government?Remote,71.67,36.6,non-smoker,NaN,NaN,NaN,NaN,0,government,remote
1,1459,"F, 30",0,0,0,Remote?private_sector,107.95,30.4,quit,NaN,NaN,NaN,NaN,0,private_sector,remote
2,7678,"F, 51",1,0,1,government?Remote,76.49,42.1,active_smoker,NaN,NaN,NaN,NaN,0,government,remote
3,34943,"F, 54",0,0,1,government?City,113.98,57.3,quit,NaN,NaN,NaN,NaN,0,government,city
4,17741,"M, 27",0,0,0,private_sector?City,70.60,26.7,active_smoker,NaN,NaN,NaN,NaN,0,private_sector,city


In [8]:
test.head()

,id,sex and age,high_BP,heart_condition_detected_2017,married,job_status and living_area,average_blood_sugar,BMI,smoker_status,TreatmentA,TreatmentB,TreatmentC,TreatmentD,job_status,living_area
0,33327,"F, 36",0.0,0.0,1.0,private_sector?Remote,76.05,33.4,active_smoker,NaN,NaN,NaN,NaN,private_sector,remote
1,839,"F, 40",0.0,0.0,1.0,City?government,73.77,30.1,non-smoker,NaN,NaN,NaN,NaN,government,city
2,11127,"M, 59",0.0,0.0,1.0,business_owner?Remote,62.95,30.8,None,NaN,NaN,NaN,NaN,business_owner,remote
3,20768,"33, F",0.0,0.0,1.0,private_sector?City,68.81,36.5,quit,NaN,NaN,NaN,NaN,private_sector,city
4,37774,"F, 22",0.0,0.0,0.0,private_sector?City,122.89,30.8,active_smoker,NaN,NaN,NaN,NaN,private_sector,city


In [ ]:
profile = train.profile_report(title='Medical Record Profiling Report')
profile.to_file(output_file="train_data_summary.html")

C:\Users\thien\Anaconda3\lib\site-packages\pandas_profiling\model\correlations.py:124: UserWarning: There was an attempt to calculate the cramers correlation, but this failed.
To hide this warning, disable the calculation
(using `df.profile_report(correlations={"cramers": False}`)
If this is problematic for your use case, please report this as an issue:
https://github.com/pandas-profiling/pandas-profiling/issues
(include the error message: 'The internally computed table of expected frequencies has a zero element at (0, 3).')
  correlation_name=correlation_name, error=error


In [ ]:
train.profile_report(style={'full_width':True})

C:\Users\thien\Anaconda3\lib\site-packages\pandas_profiling\model\correlations.py:124: UserWarning: There was an attempt to calculate the cramers correlation, but this failed.
To hide this warning, disable the calculation
(using `df.profile_report(correlations={"cramers": False}`)
If this is problematic for your use case, please report this as an issue:
https://github.com/pandas-profiling/pandas-profiling/issues
(include the error message: 'The internally computed table of expected frequencies has a zero element at (0, 3).')
  correlation_name=correlation_name, error=error


In [ ]:
test.profile_report(style={'full_width':True})

C:\Users\thien\Anaconda3\lib\site-packages\pandas_profiling\model\correlations.py:124: UserWarning: There was an attempt to calculate the cramers correlation, but this failed.
To hide this warning, disable the calculation
(using `df.profile_report(correlations={"cramers": False}`)
If this is problematic for your use case, please report this as an issue:
https://github.com/pandas-profiling/pandas-profiling/issues
(include the error message: 'The internally computed table of expected frequencies has a zero element at (0, 0).')
  correlation_name=correlation_name, error=error
